# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](https://github.com/happyemanaloto/lab-neural-networks/blob/master/your-code/tttboard.jpg?raw=1)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [ ]:
import pandas as pd
import os


# !git clone https://github.com/happyemanaloto/lab-neural-networks.git
os.chdir("/content/lab-neural-networks/your-code")
print(os.getcwd())
!ls


In [ ]:
df_ttt = pd.read_csv('tic-tac-toe.csv')
df_ttt.head()

In [ ]:

# Loop through each column and get the unique values
for column in df_ttt.columns:
    print(f"Unique values in {column}: {df_ttt[column].unique()}")
# Create a dictionary of unique values for each column
# unique_values_dict = {column: df_ttt[column].unique() for column in df_ttt.columns}

# Print the dictionary
# print(unique_values_dict)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to each categorical column
categorical_columns = ['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR','class']
df_ttt[categorical_columns] = df_ttt[categorical_columns].apply(label_encoder.fit_transform)

# View the encoded DataFrame
print(df_ttt)

In [ ]:
# # Apply LabelEncoder 'class'
# categorical_columns = ['class']
# df_ttt[categorical_columns] = df_ttt[categorical_columns].apply(label_encoder.fit_transform)

# # View the encoded DataFrame
# print(df_ttt)

In [ ]:
X = df_ttt.drop('class', axis=1)
y = df_ttt['class']

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Normalize the input features (X)
X_normalized = scaler.fit_transform(X)
X_normalized_df_ttt = pd.DataFrame(X_normalized, columns=X.columns)

# View the normalized data
print("Normalized Input Features (X):")
print(X_normalized_df_ttt.head())

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [ ]:
print(X_normalized.shape)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

##### Build the model

# Lets build a sequencial model
# model = Sequential([
#    LAYER1 -> Dense(                       -> Dense networks means all neurons in one layer are connected to all neuronsof the next layer
#                    64,                    -> Number of neurons
#                    activation='relu',     -> activation function
#                    input_shape=(784,)),   -> input shape (as it is the first layer)
#                               # 784= 28*28
#
#    LAYER 2 -> Dense(64, activation='relu'),      # choosing relu instead of sigmoid, this is somewhat common
#
#    LAYER 3 (output) -> Dense(
#                              10,           -> # of Output Neurons = # of Classes
#                              activation='softmax'),   # Softmax -> Coverts a universe of real numbers into probabilities!!
#                               ])



model = Sequential()
# model.add(Dense(64, activation='relu', input_shape=(9,), kernel_regularizer=l2(0.03)))  # L2 Regularization
# model.add(Dropout(0.5))
# model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.03)))
# model.add(Dropout(0.5))
# model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.03)))
# model.add(Dropout(0.5))

model.add(Dense(64, activation='relu', input_shape=(9,)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

#model.add(...)
#model.add(...)
#model.add(...)
#model.add(...)

model.add(Dense(2, activation='softmax'))






In [ ]:
from tensorflow.keras.optimizers import Adam

# Define a custom learning rate (e.g., 0.001)
learning_rate = 0.001

# Create the Adam optimizer with the specified learning rate
optimizer = Adam(learning_rate=learning_rate)

# Compile the model with this optimizer
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Compile the model
# model.compile(optimizer=Adam(learning_rate=0.001),  # Adam optimizer
#               loss='sparse_categorical_crossentropy',  # Loss function for multi-class classification
#               metrics=['accuracy'])  # Metrics to track during training
# model.summary()


In [ ]:
y_test

In [ ]:
#model.fit(X_train, y_train, epochs=80, batch_size=32)
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.save('tic-tac-toe.keras')

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

# Load the saved model
model = tf.keras.models.load_model('tic-tac-toe.keras')

# Select a few random rows from the test dataset
random_rows = df_ttt.sample(n=5)  # Change 5 to however many rows you want to check

# Extract features and labels from the random rows
X_random = random_rows.drop('class', axis=1)
y_random = random_rows['class']

# Make predictions on these random rows
predictions = model.predict(X_random)

# Output the predictions and the actual labels
for i in range(len(random_rows)):
    print(f"Actual label: {y_random.iloc[i]}, Predicted: {np.argmax(predictions[i])}")


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# I edited the codes above

In [ ]:
# model.fit(X_train, y_train, epochs=80, batch_size=32)
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
#adjusting the architecture improved it a bit, increasing epochs with low learning_rate is effective but changing training set and test set was best